## Import

In [1]:
import pickle
from collections import deque

from env.env import *
from state.state import *
from agent.PPOAgent_ms import *
from models.CTTS import *

## Load

In [2]:
with open('../data/processed/kospi200_ffill_clean_version.pkl', 'rb') as f:
    df = pickle.load(f)

df.head(5)

,date,time,open,high,low,close,prevClose,vol
2010-02-16 09:01:00,20100216,901,207.55,207.65,207.50,207.60,207.5,3985.0
2010-02-16 09:02:00,20100216,902,207.60,207.65,207.25,207.55,207.5,5095.0
2010-02-16 09:03:00,20100216,903,207.55,207.80,207.50,207.60,207.5,2175.0
2010-02-16 09:04:00,20100216,904,207.55,207.85,207.55,207.80,207.5,1301.0
2010-02-16 09:05:00,20100216,905,207.80,208.15,207.80,208.05,207.5,3870.0


In [3]:
target_values = ['open', 'high', 'low', 'close', 
                'vol','return_5', 'return_10', 'volume_change', 'ema_5', 
                'ema_20', 'ema_cross', 'cci', 'sar', '%K', 
                # '%D', 'roc', 'rsi', 'obv', 'ad_line', 
                'bb_upper', 'bb_lower', 'bb_width', 'atr', 'gap_size']

state = State(target_values)
scaler = RobustScaler()

In [4]:
state = State(target_values)
env = FuturesEnvironment(full_df=df, 
                         date_range=("2010-02-16", "2010-07-16"), 
                         window_size=30, 
                         state_type=state, 
                         reward_ftn=reward_unrealized_pnl_diff_log, 
                         done_ftn=is_day_changed, 
                         start_budget=20000000,
                         scaler=scaler,
                         position_cap=50)

/opt/anaconda3/envs/ML/lib/python3.10/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2
/opt/anaconda3/envs/ML/lib/python3.10/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2
/opt/anaconda3/envs/ML/lib/python3.10/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2
/opt/an

Robust Scaling Completed.


In [5]:
execution_strength = 3
action_space = list(range(-execution_strength, execution_strength+1))
n_actions = len(action_space)
device = torch.device("cpu")  

In [6]:
network = MultiStatePV(input_dim=len(target_values),              # 입력 feature 수 (D)
                        agent_input_dim=2,        # agent 상태 feature 수
                        embed_dim=32,              # CNN + Transformer 임베딩 차원 (d_model)
                        kernel_size=4,            # CNN 커널 사이즈
                        stride=1,                 # CNN stride
                        action_size=n_actions, 
                        device=device,                 # positional encoding에 필요
                        agent_hidden_dim=32, 
                        agent_out_dim=32,
                        fusion_hidden_dim=64,
                        num_layers=3,           # Transformer 층 수
                        num_heads=4,            # Multi-head attention 헤드 수
                        d_ff=64,                # FFN hidden size
                        dropout=0.1)

In [7]:
agent = PPOAgent(
    action_space=action_space,
    n_actions=n_actions,
    model=network,
    value_coeff=0.5,
    entropy_coeff=0.01,
    clip_eps=0.2,
    gamma=0.99,
    lr=1e-3
)

N_ITERATIONS = 1000
N_STEPS = 50

episode_rewards = []
moving_avg_rewards = deque(maxlen=50)

for episode in range(N_ITERATIONS):
    
    memory = []
    done = False

    if episode != 0:
        state = env.conti()
    else:
        state = env.reset()

    ts_state = torch.tensor(state[0], dtype=torch.float32).unsqueeze(0)
    agent_state = torch.tensor(state[1], dtype=torch.float32).unsqueeze(0)

    state = (ts_state, agent_state)
    

    ep_reward = 0
    ep_len = 0

    for _ in range(N_STEPS):
        if done:
            break

        action, log_prob = agent.get_action(state)

        next_state, reward, done = env.step(action)

        ts_state = torch.tensor(next_state[0], dtype=torch.float32).unsqueeze(0)
        agent_state = torch.tensor(next_state[1], dtype=torch.float32).unsqueeze(0)
        next_state = (ts_state, agent_state)

        memory.append([
            state,
            torch.tensor([[action]]),
            torch.tensor([reward], dtype=torch.float32),
            next_state,
            torch.tensor([done], dtype=torch.float32),
            torch.tensor([log_prob], dtype=torch.float32)
        ])
        state = next_state
        ep_reward += reward
        ep_len += 1

    episode_rewards.append(ep_reward)
    moving_avg_rewards.append(ep_reward)

    advantage = agent.cal_advantage(memory)
    loss = agent.train(memory, advantage)

    avg_reward = np.mean(moving_avg_rewards)
    print(f"Episode {episode:3d} | Loss: {loss: .4f} | Action: {action} |Reward: {ep_reward:3.0f} | Avg(50): {avg_reward: .2f} | Len: {ep_len}")

    if episode % 50 == 0:
        print(env)

# 후에 결과 시각화
plt.plot(episode_rewards, label='Episode Reward')
plt.plot(np.convolve(episode_rewards, np.ones(50)/50, mode='valid'), label='Moving Avg (50)')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.legend()
plt.grid()
plt.title('PPO Training Performance')
plt.show()

Episode   0 | Loss:  1.1759 | Action: -1 |Reward:  -6 | Avg(50): -5.54 | Len: 50
=== Futures Trading Environment ===
⏱️  Current Timestep   : 2010-02-16 10:39:00
📈  Previous Close     : 209.90
💼  Current Position   : long (1)
📊  Execution Strength : 9/50
📉  Unrealized PnL     : -16311158.65 KRW
💰  Current Budget     : -26591341.35 KRW
💵  Rate of Return     : -132.96 %
⚖️  Avg Entry Price    : 246.15

Episode   1 | Loss:  0.2027 | Action: 2 |Reward:  -2 | Avg(50): -3.53 | Len: 50
Episode   2 | Loss:  0.6981 | Action: 2 |Reward:  -2 | Avg(50): -3.14 | Len: 50
Episode   3 | Loss:  1.4501 | Action: 3 |Reward:  -4 | Avg(50): -3.27 | Len: 50
Episode   4 | Loss:  0.3128 | Action: 2 |Reward:  -2 | Avg(50): -3.05 | Len: 50
Episode   5 | Loss:  1.0103 | Action: 2 |Reward:  -3 | Avg(50): -3.06 | Len: 50
Episode   6 | Loss: -1.2131 | Action: 2 |Reward:  -0 | Avg(50): -2.69 | Len: 16
Episode   7 | Loss: -0.1878 | Action: 2 |Reward:  -0 | Avg(50): -2.41 | Len: 50
Episode   8 | Loss:  0.2874 | Action

StopIteration: 